In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/Shareddrives/NLP\ project/Baseline

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shareddrives/NLP project/Baseline


### **Dataset**

This is a variable!! So always upload it in the form of a CSV file with each row having

```row: intent_name,example_text```

This file must be uploaded as ```current.csv```

In [ ]:
import csv
import numpy as np

In [ ]:
intents,rev_intents = {},{}
test_X,test_Y,train_X,train_Y = [],[],[],[]
intent_counter = 1
with open('current.csv') as csvfile:
    data_reader = csv.DictReader(csvfile)
    for row in data_reader:
        intent = row['intent']
        example = row['example']
        if len(example.split())>300:
          continue
        # perform pre processing on example here // todo
        intent_encode = intents.get(intent,False)
        if not intent_encode:
            intents[intent] = intent_counter
            rev_intents[intent_counter] = intent
            intent_encode = intent_counter
            intent_counter+=1
        ''' Split data into train and test with P(Train) = 0.7'''
        if np.random.binomial(1,0.7):
            train_X.append(example)
            train_Y.append(intent_encode)
        else:
            test_X.append(example)
            test_Y.append(intent_encode)
print(len(test_Y),len(train_Y))

18079 42247


In [ ]:
print(train_X[2])

Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be "up" for this movie.


### BERT Model

In [ ]:
%%capture
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )
model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
def word_emb(text):
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)

  token_vecs = []

  for token in token_embeddings:
      cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
      token_vecs.append(cat_vec)
  return token_vecs

In [ ]:
def sent_emb(text):
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]

  token_vecs = hidden_states[-2][0]

  sentence_embedding = torch.mean(token_vecs, dim=0)

  return sentence_embedding

In [ ]:
emb_train_X = np.array([sent_emb(sent).numpy() for sent in train_X])
emb_test_X = np.array([sent_emb(sent).numpy() for sent in test_X])

print(emb_train_X.shape)
print(emb_test_X.shape)

NameError: ignored

### **Multiclass classification**

### **With sentence embeddings**

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(emb_train_X,train_Y)

# Predict Class
pred_Y = classifier.predict(emb_test_X)

# Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_Y,pred_Y)
print(accuracy)

0.8039078855547802


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_Y,pred_Y))

              precision    recall  f1-score   support

           1       0.96      0.82      0.88      1070
           2       0.14      0.56      0.22        18
           3       0.69      0.75      0.72       124
           4       0.36      0.65      0.46        23
           5       0.86      0.92      0.89        52
           6       1.00      0.44      0.62         9
           7       0.31      0.52      0.39        52
           8       0.83      0.83      0.83         6
           9       0.79      1.00      0.88        44
          10       0.57      1.00      0.73         4
          11       0.61      0.92      0.73        12
          12       0.50      0.83      0.62         6
          13       0.00      0.00      0.00         2
          14       0.38      0.75      0.50         4
          15       0.27      1.00      0.43         3
          16       0.00      0.00      0.00         3
          17       0.00      0.00      0.00         1

    accuracy              

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
text = '''
Hi everyone,

We have graded all weekly quizzes and programming assignments. Also, we have handled all the cribs related to these tasks.

So, now we are freezing the marks for quizzes and PAs, and we will not accept any requests/queries/cribs regarding these.
'''
print(rev_intents[classifier.predict([sent_emb(text).numpy()])[0]])

atis_ground_service


In [ ]:
print(rev_intents)

{1: 'atis_flight', 2: 'atis_flight_time', 3: 'atis_airfare', 4: 'atis_aircraft', 5: 'atis_ground_service', 6: 'atis_airport', 7: 'atis_airline', 8: 'atis_distance', 9: 'atis_abbreviation', 10: 'atis_ground_fare', 11: 'atis_quantity', 12: 'atis_city', 13: 'atis_flight_no', 14: 'atis_capacity', 15: 'atis_flight#atis_airfare', 16: 'atis_meal', 17: 'atis_restriction', 18: 'atis_airline#atis_flight_no', 19: 'atis_ground_service#atis_ground_fare', 20: 'atis_airfare#atis_flight_time', 21: 'atis_cheapest', 22: 'atis_aircraft#atis_flight#atis_flight_no'}
